# install and import packages

In [2]:
!pip install numpy pyDOE2 scikit-learn lime anchor-exp tabulate river pytest

Defaulting to user installation because normal site-packages is not writeable


# Experiments 

## Functions

In [3]:
from tabulate import tabulate
import time
import csv

def get_trg_sample(index):
    print('Index: ', index)
    raw_dataset = utils.load_dataset(
        dataset_name, balance=True, dataset_folder=dataset_folder, discretize=False)
    trg = dataset.test[index]
    label = dataset.labels_test[index]
    
    trg_data = list(zip(raw_dataset.feature_names, raw_dataset.test[index]))
    
    print('Prediction:', raw_dataset.class_names[rf.predict(trg.reshape(1, -1))[0]])
    print('True:      ', raw_dataset.class_names[dataset.labels_test[index]])
    print(tabulate(trg_data))
    f = open('img/%s/%05d-instance.txt' % (dataset_name, index), 'w')
    f.write(tabulate(trg_data))
    f.close()
    with open('img/%s/%05d-instance.csv' % (dataset_name, index), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([['feature', 'value']])
        writer.writerows(trg_data)
    return trg, label
    
i = 444
get_trg_sample(444)
get_trg_sample(542)

    # raw_dataset2 = utils.load_csv_dataset(
    #     os.path.join(dataset_folder, 'recidivism/Data_1980.csv'), 16,
    #     discretize=False,
    #     features_to_use=None, balance=True)
    # trg_data2 = list(zip(raw_dataset2.feature_names, raw_dataset2.test[index]))
    # print(tabulate(trg_data2))

Index:  444


NameError: name 'utils' is not defined

In [ ]:
import explanation
import lime_tabular

def lime_original(trg, trg_label):
    lime_explainer = lime_tabular.LimeTabularExplainer(
        dataset.train,
        feature_names=dataset.feature_names,
        class_names=dataset.class_names,
        discretize_continuous=False)
    lime_exp = lime_explainer.explain_instance(
        trg, rf.predict_proba, num_features=5, top_labels=1)
    # lime_exp.show_in_notebook(show_table=True, show_all=True)
    weights = [0] * len(dataset.feature_names)
    for t in lime_exp.local_exp[trg_label]:
        weights[t[0]] = t[1] * (trg_label * 2 - 1)
    plot_weights(weights, dataset.feature_names)

In [ ]:
from anchor import anchor_tabular

def anchor_original(trg, threshold=0.85):
    anchor_explainer = anchor_tabular.AnchorTabularExplainer(
        dataset.class_names,
        dataset.feature_names,
        dataset.train,
        dataset.categorical_names)
    anchor_exp = anchor_explainer.explain_instance(
        trg, rf.predict, threshold)

    print('Threshold:  %.2f' % threshold)
    # print('Prediction:', anchor_explainer.class_names[rf.predict(trg.reshape(1, -1))[0]])
    print('Anchor:     %s' % (' AND '.join(anchor_exp.names())))
    print('Precision:  %.2f' % anchor_exp.precision())
    print('Coverage:   %.2f' % anchor_exp.coverage())

In [ ]:
import my_anchor_base
import my_anchor_tabular

# display
import matplotlib.pyplot as plt
    
import importlib
importlib.reload(my_anchor_base)
importlib.reload(my_anchor_tabular)

def new_lime(trg, threshold=0.90):
    anchor_explainer = my_anchor_tabular.AnchorTabularExplainer(
        dataset.class_names,
        dataset.feature_names,
        dataset.train,
        dataset.categorical_names)
    anchor_exp, surrogate_model = anchor_explainer.explain_instance(trg, rf.predict, threshold=threshold)
    plot_weights(
        list(surrogate_model['LogisticRegression'].weights.values()),
        dataset.feature_names,
        ' AND '.join(anchor_exp.names()),
        anchor_exp.precision(),
        anchor_exp.coverage()
    )
    

In [ ]:
def plot_weights(weights, feature_names, anchor=None, precision=None, coverage=None):
    features = feature_names
    values = weights
    abs_values = [abs(x) for x in values]
    _, sorted_features, sorted_values = zip(*sorted(zip(abs_values, features, values), reverse=False)[-5:])
    plt.figure()
    color = ['#32a852' if sorted_values[i] > 0 else '#cf4529' for i in range(len(sorted_values))]
    plt.barh(sorted_features, sorted_values, color=color)
    anchor_str = ''
    
    if anchor != None:
        plt.title('%s\nwith Precision %.2f and Coverage %.2f' % (anchor, precision, coverage))
    for i in range(len(sorted_features)):
        plt.text(sorted_values[i], sorted_features[i], round(sorted_values[i], 5))
    plt.show

## Generating Explanations

In [ ]:
import utils
import os

dataset_folder = 'datasets/'
dataset_name = 'recidivism'
# dataset_name = 'adult'
# dataset_name = 'diabetes'
# dataset_name = 'default'
# dataset_name = 'lending'

dataset = utils.load_dataset(
    dataset_name, balance=True, dataset_folder=dataset_folder, discretize=True)

In [ ]:
import sklearn.ensemble

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)

rf.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(
    dataset.labels_train, rf.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(
    dataset.labels_test, rf.predict(dataset.test)))

In [ ]:
import numpy as np
import random

importlib.reload(my_anchor_base)
importlib.reload(my_anchor_tabular)

i = random.randint(10, dataset.test.shape[0])

# adult
# 730, 956, 1196, 1522
# i = 1443 # --- Capital Gain = 2

# recidivism 
# 444, 542
# 72
# i = 80 ???
# i = 444
i = 542
trg, trg_label = get_trg_sample(i)

In [ ]:
lime_original(trg, trg_label)
img_file = 'img/%s/%05d-LIME.png' % (dataset_name, i)
plt.savefig(img_file, bbox_inches='tight')

In [ ]:
for t in [0.70, 0.75, 0.80]:
    new_lime(trg, threshold=t)
    img_file = 'img/%s/%05d-%03d.png' % (dataset_name, i, t * 100)
    plt.savefig(img_file, bbox_inches='tight')

In [ ]:
new_lime(trg, threshold=0.75)

In [ ]:
new_lime(trg, threshold=0.80)

In [ ]:
anchor_original(trg, threshold=0.80)